In [1]:
from ipywidgets import interact_manual
from vpython import canvas, helix, rate, scene, sphere, vector

<IPython.core.display.Javascript object>

In [35]:
@interact_manual(m=(0.1, 20), k=(0.1, 20))
def animate_spring(t_max=10.0, m=1.0, k=9.81):
    spring_length = 1.0
    canvas(forward=vector(0.0, -1.0, 0.0))
    pivot_pos = vector(0.0, 0.0, 0.5)
    mass_pos = vector(0.0, 0.0, -spring_length)
    spring_axis_orig = mass_pos - pivot_pos
    spring = helix(pos=pivot_pos, axis=spring_axis_orig,
                   radius=0.2, width=0.1)
    pivot = sphere(pos=pivot_pos, radius=0.1)
    mass = sphere(pos=mass_pos, radius=0.2*m**(1/3))
    mass.v = vector(0.0, 0.0, 0.0)
    g = 9.81
    t = 0.0
    delta_t = 0.01
    while t <= t_max:
        rate(int(1.0/delta_t))
        grav_force = m*g*vector(0.0, 0.0, -1.0)
        spring_force = k*(spring_axis_orig - spring.axis)
        mass.v += delta_t*(grav_force + spring_force)/m
        mass.pos += delta_t*mass.v
        spring.axis = mass.pos - pivot.pos
        t += delta_t

interactive(children=(FloatSlider(value=10.0, description='t_max', max=30.0, min=-10.0), FloatSlider(value=1.0…